# <center><font color='red'> TME 2 - Programmation Dynamique </font></center>

Ce TME a pour objectif d'expérimenter les modèles algorithmes de programmation dynamique sur un MDP classique de type GridWorld. <br>
Nous utiliserons au cours des TME de RL la plateforme en python gym (de open-ai ). <br>
Pour l'installer, tapez la commande: <font color="green"> pip3 install gym --proxy proxy:3128 --user </font>

# 1. Gym et GridWorld

L'environnement gridworld-v0 pour gym (fourni dans le chier associé au TME) est une tâche de RL où un agent (point bleu) doit récolter des éléments jaunes dans un labyrinthe 2D et terminer sur une case verte, tout en évitant les cases roses (non terminales) et rouges (terminales). Ci-dessous quelques fonctions utiles pour les environnements :

<ul>
    <li> initialiser un environnement : env = gym.make('gridworld-v0') puis env.reset() </li>
    <li> env.action_space permet de connaître l'ensemble des actions possibles; </li>
    <li> obs,reward,done,info = env.step(action) permet de jouer l'action passée en argument : obs contient le nouvel état, reward la récompense associée à l'action, done un booléen pour savoir si le jeu est fini; </li>
    <li> env.render() permet de visualiser le jeu (si env.verbose est à true); </li>
</ul>

Pour le cas du gridworld la fonction env.getMDP() retourne un couple (states,P), où:

<ul>
    <li> states est un dictionnaire associant les observations à des numéros d'états (states[gridworld.GridworldEnv.state2str(observation)] contient le numéro d'état d'une observation obs avec observation un tableau numpy représentant la carte); </li>
    <li> P correspond à l'automate du MDP de la tâche : chaque clé est un état, chaque valeur un dictionnaire; pour ce 2ème dictionnaire, chaque clé est une action et la valeur une liste de tuples correspondant à la transition associée (proba de la transition,état destination,reward,done), où done est vrai si l'état de destination est un état terminal. Ainsi, P[state][action] permet de connaître
pour un état et une action la liste des états atteignables avec leur probabilité et la récompense associée. </li>
</ul>

Télécharger sur le site le fichier source associé au TME. Le répertoire gridworld contient l'environnement du labyrinthe; le fichier randomAgent.py présente des exemples d'utilisation de l'environnement et de gym et en particulier un agent aléatoire.

Dans l'environnement gridworld il est possible de charger différentes cartes de problème par la fonction setPlan("gridworldPlan/planX.txt",{0:-0.001,3:1,4:1,5:-1,6:-1}) prenant en argument le fichier de la carte à charger et une liste des récompenses associées aux différents types de cases du jeu : 0 correspond à une case vide, 1 correspond à un mur (pas de reward associé car impossible de s'y déplacer), 2 correspond au joueur, 3 correspond à une case verte, 4 une case jaune, 5 une case rouge et 6 une case rose. <br>
Dans le module fourni, vous trouverez différentes cartes de jeu (X de 0 à 10), de difficulté variable, que vous devrez tester au cours du TME.

# 2. Travail demandé

Codez les algorithmes de policy iteration et de value iteration vus en cours et testez les sur l'environnement. Discutez des résultats obtenus sur les différentes cartes/configurations (vous pouvez aussi en imaginer d'autres). Vous pouvez aussi faire varier le paramètre de discount pour en observer l'impact.

In [1]:
import matplotlib
import matplotlib.pyplot as plt

#matplotlib.use("TkAgg")
import pandas as pd
import gym
import gridworld
from gym import wrappers, logger
import numpy as np
import copy
from tqdm import tqdm

In [2]:
class RandomAgent(object):
    """The world's simplest agent!"""

    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation, reward, done):
        return self.action_space.sample()

In [19]:
class PolicyIteration():
    
    def __init__(self, action_space, statedic, mdp, eps=1e-10, gamma=0.99):
        self.statedic = statedic
        self.mdp = mdp
        self.action_space = action_space
        self.eps = eps
        self.gamma = gamma
        
    def act(self, obs, reward, done):
        nb_s = len(self.statedic) # Nombre d'états
        
        # dict avec comme clé les différents états et comme valeur les récompenses initialisées aléatoirement
        # V = dict(zip(self.statedic, np.random.random(nb_s)))

        # tab de tab des états possibles
        states = [s for s in self.statedic]
        
        # dict des états associés initialement à la valeur None
        pi = {s: np.random.randint(0, self.action_space.n) for s in states}
        V = {s: 0 for s in states}

        ft = True # First Time
        ft2 = True

        while ft:
            pi_copy = copy.deepcopy(pi)
            while ft2:
                V_copy = copy.deepcopy(V)
                
#                 for state, dico in self.mdp.items(): # parcours de tous les états
#                     V_inter = 0
#                     action = pi[state]
                    
                    V_inter = [np.sum([p*(rew+self.gamma*V_copy[s_dest]) 
                        for p, s_dest, rew, _ in self.mdp[state][action]])
                              for action, tuples in dico.items()] 
                    
#                     for p, s_dest, rew, _ in dico[action]:
#                         V_inter += ( p * (rew + self.gamma * V_copy[s_dest]) )
#                     #print("V inter : ", V_inter)
#                     V[state] = V_inter

                for s, dico in self.mdp.items():

                    action = pi[s]
                    V[s] = np.sum([p * (rew + self.gamma * V_copy[s_dest] ) for p, s_dest, rew, _ in self.mdp[s][action] ])

#                 if ( np.linalg.norm(np.array(list(V_copy.values())) - np.array(list(V.values())))) <= self.eps:
#                     ft2 = False

                if np.sum(np.fabs(np.array(list(V_copy.values())) - np.array(list(V.values())))) <= self.eps:
                    ft2 = False

            
            for state, dico in self.mdp.items(): # parcours de tous les états
                
                V_inter = [np.sum([p*(rew+self.gamma*V[s_dest]) 
                            for p, s_dest, rew, _ in tuples]) 
                                for action, tuples in dico.items() ]

                #print("ll : ", len(V_inter))
                pi[state] = np.argmax(V_inter)
                
                
            if pi_copy == pi:
                ft = False
                

            
        return pi[gridworld.GridworldEnv.state2str(obs)]

In [50]:
class ValueIteration():
    
    def __init__(self, action_space, statedic, mdp, eps=0.001, gamma=0.99):
        self.statedic = statedic
        self.mdp = mdp
        self.action_space = action_space
        self.eps = eps
        self.gamma = gamma
        
    def act(self, obs, reward, done):
        nb_s = len(self.statedic) # Nombre d'états
        V = dict(zip(self.statedic, np.random.random(nb_s)))
        states = [s for s in self.statedic]
        pi = {s: None for s in states}
        ft = True # First Time
        i = 0
    
        while ft:
            V_copy = copy.deepcopy(V)
            for state, dico in self.mdp.items(): # parcours de tous les états
                
                V_inter = [np.sum([p*(rew+self.gamma*V_copy[s_dest]) 
                        for p, s_dest, rew, _ in tuples]) 
                            for action, tuples in dico.items() ]
                
                V[state] = np.max(V_inter)
                
            if np.linalg.norm(np.array(list(V.values())) - np.array(list(V_copy.values()))) <= self.eps:
                ft = False
    
        for state, dico in self.mdp.items(): # parcours de tous les états
                
            V_inter = [np.sum([p*(rew+self.gamma*V[s_dest]) 
                        for p, s_dest, rew, _ in self.mdp[state][action]]) 
                            for action, tuples in dico.items() ]

            pi[state] = np.argmax(V_inter)
    
   
            
        return pi[gridworld.GridworldEnv.state2str(obs)]

In [51]:
def play(agentType="random", worldNumber=0):
    # env.action_space : ens des actions possibles
    # env.action_space.n : nombre d'actions possibles
    # env.observation_space : ens des états possibles
    # env.observation_space : nombre d'états possibles

    env = gym.make("gridworld-v0") # Init un environnement

    # setPlan(arg1, arg2)
    # arg1 : fichier de la carte à charger
    # arg2 : liste de récompenses associées aux différents types de cases du jeu 
    env.setPlan("gridworldPlans/plan"+str(worldNumber)+".txt", {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1})

    env.verbose = True

    statedic, mdp = env.getMDP()  

    if agentType == "random":
        agent = RandomAgent(env.action_space)
        
    elif agentType == "policy":
        agent = PolicyIteration2(env.action_space, statedic, mdp)
        
    elif agentType == "value":
        agent = ValueIteration(env.action_space, statedic, mdp)
    
    else:
        agent = RandomAgent(env.action_space)
        print("Agent inconnu : agent random par défaut")
    

    # Faire un fichier de log sur plusieurs scenarios
    outdir = 'gridworld-v0/random-agent-results'
    envm = wrappers.Monitor(env, directory=outdir, force=True, video_callable=False)


    countRewards = []

    episode_count = 1000
    reward = 0
    done = False
    rsum = 0
    FPS = 0.001

    for i in tqdm(range(episode_count)):

        obs = envm.reset()
        env.verbose = (i % 100 == 0 and i > 0)  # afficher 1 episode sur 100
#         if env.verbose:
#             env.render(FPS)
#             env.render(mode="human")
        j = 0
        rsum = 0
        while True:
            action = agent.act(obs, reward, done)
            obs, reward, done, _ = envm.step(action)
            rsum += reward
            j += 1
#             if env.verbose:
#                 env.render(FPS)
            if done:
                #print("Episode : " + str(i) + " rsum=" + str(rsum) + ", " + str(j) + " actions")
                countRewards.append(rsum)
                break

    np.save("rewards_gridworld_"+str(worldNumber)+"_"+agentType+".npy", countRewards)          
    print("Mean & std : ", np.mean(countRewards), np.std(countRewards))
    print("done")
    env.close()
    
    return countRewards

# 3. Quelques résultats

In [52]:
cr = play("policy", 0)

100%|██████████| 1000/1000 [00:01<00:00, 667.11it/s]

Mean & std :  -1.096846 0.10000114141348596
done


In [164]:
a = np.load("rewards_gridworld_0_policy.npy")
np.mean(a)

0.5183289999999999

In [272]:
a = {'s' : 23, 'd' : 37}
b = {'s' : 23, 'd' : 27}
print(b)
b = copy.deepcopy(a)
print(b)

{'s': 23, 'd': 27}
{'s': 23, 'd': 37}


In [258]:
a = 12
print(np.sum(a))

12


<font color="green"> Le temps de convergence de l'algorithme **Value Iteration** est plus rapide que celle de **Policy Iteration** </font>